In [208]:
import pandas as pd
import dash
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import plotly.express as px
import json
import geojson
import math


df_dejavnosti = pd.read_csv("data/podatki_po_dejavnostih.csv", delimiter=",", encoding="windows-1250")
df_odjemalci = pd.read_csv("data/podatki_vrsta_odjemalca.csv", delimiter=",", encoding="windows-1250")

In [209]:
df_odjemalci_backup = df_odjemalci.copy()

#Tukaj vzamemo samo podatke za VSE SKUPAJ porabo (brez gospodinjstva in poslovnih objektov)
df = df_odjemalci_backup[::+3]
#Tukaj izbrišem podatke za SLOVENIJO in Neznano vrstice
df = df[1:-1]

#Priprava podatkov za uporabo
df = df.drop("VRSTA ODJEMALCA", axis=1)
obcine = {df["OBČINE"][i]:i for i in df.index}

vse_2020 = dict(df[df.columns[1:13]].sum(axis=1))

dict_porab = dict([(k,vse_2020[k]) for k in vse_2020])

#V koncem dictu so shranjene OBČINE (ključ) z vrednostmi, ki so LISTI, 
# ki prikazujejo porabo za vsako občino za vsako leto posebaj
koncni_dict = dict()
for k,v in obcine.items():
    koncni_dict[k] = dict_porab[v]


In [210]:
def izrisi_odjemalci_Slovenija(odjemalci, leto):
    df_odjemalci = odjemalci.copy()
    df=df_odjemalci[::+3]
    df = df[1:-1]
    df = df.drop("VRSTA ODJEMALCA", axis=1)
    obcine = {df["OBČINE"][i]:i for i in df.index}

    dict_porab = dict()
    if leto == 2020:
        vse_2020 = dict(df[df.columns[1:13]].sum(axis=1))
        dict_porab = dict([(k,[vse_2020[k]]) for k in vse_2020])
    elif leto == 2021:
        vse_2021 = dict(df[df.columns[13:25]].sum(axis=1))
        dict_porab = dict([(k,[vse_2021[k]]) for k in vse_2021])

    koncni_dict = dict()
    for k,v in obcine.items():
        koncni_dict[k] = dict_porab[v]
    
    return pd.DataFrame.from_dict(koncni_dict).transpose()

In [211]:
def izrisi_odjemalci_gospodinjstva(odjemalci, leto):
    df_odjemalci = odjemalci.copy()
    df=df_odjemalci[1::+3]
    df = df[1:-1]
    df = df.drop("VRSTA ODJEMALCA", axis=1)
    obcine = {df["OBČINE"][i]:i for i in df.index}

    dict_porab = dict()
    if leto == 2020:
        vse_2020 = dict(df[df.columns[1:13]].sum(axis=1))
        dict_porab = dict([(k,[vse_2020[k]]) for k in vse_2020])
    elif leto == 2021:
        vse_2021 = dict(df[df.columns[13:25]].sum(axis=1))
        dict_porab = dict([(k,[vse_2021[k]]) for k in vse_2021])

    koncni_dict = dict()
    for k,v in obcine.items():
        koncni_dict[k] = dict_porab[v]

 
    return pd.DataFrame.from_dict(koncni_dict).transpose()


In [212]:
def izrisi_odjemalci_poslovni_objekti(odjemalci, leto):
    df_odjemalci = odjemalci.copy()
    df=df_odjemalci[2::+3]
    df = df[1:-1]
    df = df.drop("VRSTA ODJEMALCA", axis=1)
    obcine = {df["OBČINE"][i]:i for i in df.index}

    dict_porab = dict()
    if leto == 2020:
        vse_2020 = dict(df[df.columns[1:13]].sum(axis=1))
        dict_porab = dict([(k,[vse_2020[k]]) for k in vse_2020])
    elif leto == 2021:
        vse_2021 = dict(df[df.columns[13:25]].sum(axis=1))
        dict_porab = dict([(k,[vse_2021[k]]) for k in vse_2021])

    koncni_dict = dict()
    for k,v in obcine.items():
        koncni_dict[k] = dict_porab[v]

    return pd.DataFrame.from_dict(koncni_dict).transpose()

In [213]:
#Izris podatkov PRIMERI (2020 in 2021)
df = izrisi_odjemalci_Slovenija(df_odjemalci, 2020)
#izrisi_odjemalci_gospodinjstva(df_odjemalci, 2020)
#izrisi_odjemalci_poslovni_objekti(df_odjemalci, 2020)

In [214]:
map_df1 = gpd.read_file('slovenija_map/obcine/obcine.shp')
map_df1 = map_df1.to_crs("WGS84")
map_df1 = map_df1.rename({'NAZIV': 'District'}, axis = 'columns')
map_df1 = map_df1.drop(columns = ['EID_OBCINA',  'SIFRA', 'NAZIV_DJ', 'OZNAKA_MES', 'DATUM_SYS'])
map_df1 = map_df1.sort_values('District')

list(map_df1["District"])

['Ajdovščina',
 'Ankaran',
 'Apače',
 'Beltinci',
 'Benedikt',
 'Bistrica ob Sotli',
 'Bled',
 'Bloke',
 'Bohinj',
 'Borovnica',
 'Bovec',
 'Braslovče',
 'Brda',
 'Brezovica',
 'Brežice',
 'Cankova',
 'Celje',
 'Cerklje na Gorenjskem',
 'Cerknica',
 'Cerkno',
 'Cerkvenjak',
 'Cirkulane',
 'Destrnik',
 'Divača',
 'Dobje',
 'Dobrepolje',
 'Dobrna',
 'Dobrova-Polhov Gradec',
 'Dobrovnik',
 'Dol pri Ljubljani',
 'Dolenjske Toplice',
 'Domžale',
 'Dornava',
 'Dravograd',
 'Duplek',
 'Gorenja vas-Poljane',
 'Gorišnica',
 'Gorje',
 'Gornja Radgona',
 'Gornji Grad',
 'Gornji Petrovci',
 'Grad',
 'Grosuplje',
 'Hajdina',
 'Hodoš',
 'Horjul',
 'Hoče-Slivnica',
 'Hrastnik',
 'Hrpelje-Kozina',
 'Idrija',
 'Ig',
 'Ilirska Bistrica',
 'Ivančna Gorica',
 'Izola',
 'Jesenice',
 'Jezersko',
 'Juršinci',
 'Kamnik',
 'Kanal',
 'Kidričevo',
 'Kobarid',
 'Kobilje',
 'Komen',
 'Komenda',
 'Koper',
 'Kostanjevica na Krki',
 'Kostel',
 'Kozje',
 'Kočevje',
 'Kranj',
 'Kranjska Gora',
 'Križevci',
 'Krško',
 '

In [215]:
df = df.sort_index()
list(df.index)

['Ajdovščina',
 'Ankaran/Ancarano',
 'Apače',
 'Beltinci',
 'Benedikt',
 'Bistrica ob Sotli',
 'Bled',
 'Bloke',
 'Bohinj',
 'Borovnica',
 'Bovec',
 'Braslovče',
 'Brda',
 'Brezovica',
 'Brežice',
 'Cankova',
 'Celje',
 'Cerklje na Gorenjskem',
 'Cerknica',
 'Cerkno',
 'Cerkvenjak',
 'Cirkulane',
 'Destrnik',
 'Divača',
 'Dobje',
 'Dobrepolje',
 'Dobrna',
 'Dobrova - Polhov Gradec',
 'Dobrovnik/Dobronak',
 'Dol pri Ljubljani',
 'Dolenjske Toplice',
 'Domžale',
 'Dornava',
 'Dravograd',
 'Duplek',
 'Gorenja vas - Poljane',
 'Gorišnica',
 'Gorje',
 'Gornja Radgona',
 'Gornji Grad',
 'Gornji Petrovci',
 'Grad',
 'Grosuplje',
 'Hajdina',
 'Hodoš/Hodos',
 'Horjul',
 'Hoče - Slivnica',
 'Hrastnik',
 'Hrpelje - Kozina',
 'Idrija',
 'Ig',
 'Ilirska Bistrica',
 'Ivančna Gorica',
 'Izola/Isola',
 'Jesenice',
 'Jezersko',
 'Juršinci',
 'Kamnik',
 'Kanal',
 'Kidričevo',
 'Kobarid',
 'Kobilje',
 'Komen',
 'Komenda',
 'Koper/Capodistria',
 'Kostanjevica na Krki',
 'Kostel',
 'Kozje',
 'Kočevje',
 'K

In [216]:
map_df = gpd.read_file('slovenija_map/obcine/obcine.shp')
map_df = map_df.to_crs("WGS84")
map_df = map_df.rename({'NAZIV': 'District'}, axis = 'columns')
map_df = map_df.drop(columns = ['EID_OBCINA',  'SIFRA', 'NAZIV_DJ', 'OZNAKA_MES', 'DATUM_SYS'])
map_df = map_df.sort_values('District')


map_df.loc[map_df["District"] == "Ljubljana"]

,District,geometry
190,Ljubljana,"POLYGON ((14.66923 45.98865, 14.66929 45.98852..."


In [217]:
list(df[0].values)

[93181950,
 19112313,
 10051617,
 24589761,
 6607746,
 3319459,
 38934450,
 6787877,
 28971836,
 19832338,
 18568541,
 16420394,
 18503852,
 33368066,
 90200634,
 5074799,
 330313558,
 43976033,
 49833430,
 39151682,
 5421889,
 6658585,
 6993382,
 25921110,
 1863100,
 11794309,
 8950588,
 18678705,
 5563918,
 21410910,
 13698882,
 274321396,
 7103397,
 35953019,
 16585460,
 29380393,
 11647809,
 8839709,
 51104644,
 7242415,
 7445634,
 4443971,
 93398833,
 14768766,
 1379943,
 12369414,
 100426518,
 102165022,
 25358462,
 73845109,
 22918383,
 119220962,
 68257700,
 70333379,
 66710498,
 1848845,
 5612012,
 250104598,
 149936812,
 26034977,
 17968471,
 1104508,
 14082858,
 42924890,
 300401924,
 8419496,
 2286668,
 9492197,
 71868594,
 304913993,
 34504928,
 15234835,
 259695031,
 16287644,
 3863708,
 69323792,
 42540741,
 100193172,
 87967919,
 1629800536,
 23843051,
 59173442,
 10075177,
 85647507,
 8818933,
 26029489,
 4824009,
 21351160,
 4639286,
 9698934,
 4543987,
 488724263,
 1

In [218]:
def uredi_data_prebivalci(prebivalci_data):
    df = prebivalci_data.copy()
    df = df.rename({'OBČINE': 'District'}, axis = 'columns')
    df = df.drop(df.index[0])

    df.index = map_df.index
    df["District"] = map_df["District"]

    df["2020_skupaj"] = ((df["2020H1 Starost - SKUPAJ"] + df["2020H2 Starost - SKUPAJ"]) / 2).astype(int)
    df["2021_skupaj"] = ((df["2021H1 Starost - SKUPAJ"] + df["2021H2 Starost - SKUPAJ"]) / 2).astype(int)
    df = df.drop(columns = ["2021H1 Starost - SKUPAJ",  '2021H2 Starost - SKUPAJ', '2020H1 Starost - SKUPAJ', '2020H2 Starost - SKUPAJ'])

    return df

In [222]:
df_prebivalci = pd.read_csv("data/podatki_starost_prebivalcev.csv", delimiter=",", encoding="windows-1250")

df_urejeni = uredi_data_prebivalci(df_prebivalci)

d1 = izrisi_odjemalci_Slovenija(df_odjemalci, 2020)
d2 = izrisi_odjemalci_gospodinjstva(df_odjemalci, 2020)
d3 = izrisi_odjemalci_poslovni_objekti(df_odjemalci, 2020)


def izracun_normalizacije_Slovenija(df, urejeni, mapdf):
    df.index = mapdf.index
    df.insert(0,'District', mapdf["District"])
    df["Normalizacija_2020"] = round(df[0] / urejeni["2020_skupaj"],2)
    df["Normalizacija_2021"] = round(df[0] / urejeni["2021_skupaj"],2)
    return df

def izracun_normalizacije_gospodinjstva(df, urejeni, mapdf):
    df.index = mapdf.index
    df.insert(0,'District', mapdf["District"])
    df["Normalizacija_2020"] = round(df[0] / urejeni["2020_skupaj"],2)
    df["Normalizacija_2021"] = round(df[0] / urejeni["2021_skupaj"],2)
    return df

def izracun_normalizacije_poslovni_objekti(df, urejeni, mapdf):
    df.index = mapdf.index
    df.insert(0,'District', mapdf["District"])
    df["Normalizacija_2020"] = round(df[0] / urejeni["2020_skupaj"],2)
    df["Normalizacija_2021"] = round(df[0] / urejeni["2021_skupaj"],2)
    return df

izracun_normalizacije_Slovenija(d1, df_urejeni, map_df)
#izracun_normalizacije_gospodinjstva(d2, df_urejeni, map_df)
#izracun_normalizacije_poslovni_objekti(d3, df_urejeni, map_df)

,District,0,Normalizacija_2020,Normalizacija_2021
64,Ajdovščina,93181950,4767.80,4724.77
63,Ankaran,19112313,5902.51,5871.68
147,Apače,10051617,2841.85,2826.66
65,Beltinci,24589761,3054.25,3039.90
29,Benedikt,6607746,2528.80,2505.78
...,...,...,...,...
1,Železniki,38035966,5674.47,5684.65
116,Žetale,2495314,1904.82,1915.05
10,Žiri,24799402,5018.09,4975.80
131,Žirovnica,18732777,4248.76,4189.84
